# Overview

Here, we will demonstrate how to carry out geometry optimizations using ASE. For this, we need to pick an energy model, which in ASE is called a `Calculator`. The `Calculator` is the simulation engine that, given an atomic configuration, will return the energy of the system (as well as forces and, for solids, the unit cell stresses).

For this exercise, we will mainly use a "machine learning potential". We will cover machine learning potentials in greater detail later in the course, but it is a machine learning model trained to reproduce density functional theory energy, forces, and stresses at a low computational cost. The model we will use in this exercise is called TensorNet-MatPES-r2SCAN as reported in https://arxiv.org/abs/2503.04070.

For additional details on how to carry out geometry optimizations in ASE, refer to the [Structure Optimization](https://ase-lib.org/ase/optimize.html) section of the documentation.


# Setup


In [1]:
%uv pip install ase@git+https://gitlab.com/ase/ase.git

Using Python 3.13.7 environment at: /Users/ct5868/personal/cbe423/.venv
   Updating https://gitlab.com/ase/ase.git (HEAD)                  
   Updating https://gitlab.com/ase/ase.git (HEAD)          
   Updating https://gitlab.com/ase/ase.git (HEAD)          
   Updating https://gitlab.com/ase/ase.git (HEAD)          
   Updating https://gitlab.com/ase/ase.git (HEAD)          
    Updated https://gitlab.com/ase/ase.git (9e022fa5b8d123bc1785145661104a4f893a
Resolved 13 packages in 683ms                                        
Audited 13 packages in 0.34ms
Note: you may need to restart the kernel to use updated packages.


For this demonstration, we also need to install a code called matcalc. This will give us the machine learning potential to use as our energy and force calculator.


In [3]:
%uv pip install 'matcalc[matgl]'

Using Python 3.13.7 environment at: /Users/ct5868/personal/cbe423/.venv
Audited 1 package in 25ms
Note: you may need to restart the kernel to use updated packages.


# Demonstration


## Preparing the `Atoms` object and `Calculator`


First, we will need to define our atomic system. We will study a porous material called a metal-organic framework, specifically MOF-5.


We can start by reading in our crystal structure with ASE and visualizing it.


In [4]:
from ase.io import read
from ase.visualize import view

original_atoms = read("mof5.cif")
view(original_atoms)

<Popen: returncode: None args: ['/Users/ct5868/personal/cbe423/.venv/bin/pyt...>

Since we want our calculations to be efficient, let's instead start by making a primitive cell using Pymatgen, which we will then convert to an ASE `Atoms` object.


In [5]:
from pymatgen.core import Structure

structure = Structure.from_file("mof5.cif", primitive=True)
atoms = structure.to_ase_atoms()

/Users/ct5868/personal/cbe423/.venv/lib/python3.13/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = parser.parse_structures(primitive=primitive)[0]


Now let's view the new `Atoms` object. You'll see that it is a primitive cell and contains fewer atoms. If you want to make it a bit more fun, go ahead and distort one of the atoms a little bit in the GUI.


In [6]:
from ase.visualize import view

view(atoms)

<Popen: returncode: None args: ['/Users/ct5868/personal/cbe423/.venv/bin/pyt...>

Now we need to define a calculator. ASE comes with many pre-made calculators (https://ase-lib.org/ase/calculators/calculators.html), and there are many external codes that ship their own ASE calculators.


We will load the TensorNet-MatPES-r2SCAN calculator use the `load_fp` function (for "load foundation potential") in matcalc.


In [4]:
from matcalc import load_fp

calc = load_fp("TensorNet-MatPES-r2SCAN-v2025.1-PES")

/Users/ct5868/personal/cbe423/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can attach the calculator to the `Atoms` object.


In [5]:
atoms.calc = calc

## Static Calculation


We now have everything we need to run a calculation. Let's start with running a static (single-point) calculation.


In [7]:
e_initial = atoms.get_potential_energy()
print(e_initial)

-834.7975463867188


It is always important to understand the units of any given code. In the case of ASE, energies are given in eV (https://ase-lib.org/ase/units.html). There are 96.49 kJ/mol per eV.

An absolute energy is meaningless. It must be referenced to something. We will come back to this.


## Geometry Optimization


The structure we loaded may not be a minimum energy structure. We must run a geometry optimization to obtain this.

ASE comes with many optimization routines. We will use BFGS.


In [8]:
from ase.optimize import BFGS

# Set up the BFGS optimizer and specify output file
opt = BFGS(atoms, trajectory="opt.traj")

In [9]:
opt.run(fmax=0.01)  # Run until the maximum force is less than 0.01 eV/Å

      Step     Time          Energy          fmax
BFGS:    0 13:45:29     -834.797546        0.343945
BFGS:    1 13:45:29     -834.838318        0.215464
BFGS:    2 13:45:29     -834.863037        0.134177
BFGS:    3 13:45:29     -834.869263        0.099389
BFGS:    4 13:45:29     -834.874878        0.063087
BFGS:    5 13:45:29     -834.877991        0.051151
BFGS:    6 13:45:29     -834.878967        0.017652
BFGS:    7 13:45:29     -834.879150        0.010641
BFGS:    8 13:45:29     -834.879211        0.009295


np.True_

The `Atoms` object is updated in place, meaning that all the properties of it are now for the optimized structure and not the initial one.


In [10]:
e_final = atoms.get_potential_energy()  # get the optimized energy
print(e_final - e_initial)  # print the energy difference

-0.0816650390625


In [11]:
atoms.get_forces()  # N_atoms x 3 matrix to describe F_x, F_y, F_z

array([[-7.17272778e-05,  4.53710556e-04,  1.09729503e-04],
       [-9.24861524e-07, -3.73899937e-04, -4.32191882e-07],
       [ 1.98643116e-04, -2.91168690e-04,  1.98897396e-04],
       [-6.99027441e-06,  3.93688679e-04, -6.29742863e-06],
       [ 2.01534625e-04, -1.87082172e-04,  4.02033329e-04],
       [-3.96044925e-06,  7.65116420e-06, -3.82363796e-04],
       [-2.03364907e-04, -2.03088828e-04, -4.27663326e-04],
       [-4.39775176e-05, -4.35225666e-05,  3.95655632e-04],
       [ 3.58283520e-04, -1.94122898e-04,  1.94191161e-04],
       [-3.93390656e-04,  1.06366351e-05, -4.11132351e-06],
       [-3.04698944e-04, -2.00720096e-05, -2.00277627e-05],
       [ 4.05013561e-04,  4.60480805e-07,  4.29223292e-07],
       [-1.16074894e-04, -6.90846145e-03,  8.39054992e-05],
       [ 1.85179524e-05,  6.78908825e-03, -7.79540278e-06],
       [-9.17067664e-05,  6.79284334e-03, -9.15925048e-05],
       [ 5.07431105e-05, -6.79925084e-03,  4.82363394e-05],
       [-8.16945540e-05,  6.27888730e-05

If want to find the $\max|\bold{F}|$ across the structure, we need to compute $|\bold{F}_i| = \sqrt{F_{x,i}^2 + F_{y,i}^2 + F_{z,i}^2}$ for each atom $i$.


In [16]:
import numpy as np

# |F_i| for all atoms i
force_mags = np.linalg.norm(atoms.get_forces(), axis=1)  # axis=1 means row-wise norm

# max|F|
max_f = force_mags.max()
print(f"max|F| = {max_f}")

max|F| = 0.00929467286914587


It looks like we found a lower energy structure. Let's take a look at the optimization trajectory now.


In [17]:
trajectory = read("opt.traj", index=":")  # index=":" means read all the frames

In [18]:
print(f"The trajectory has {len(trajectory)} frames")

The trajectory has 9 frames


In [19]:
from ase.visualize import view

view(trajectory)

<Popen: returncode: None args: ['/Users/ct5868/personal/cbe423/.venv/bin/pyt...>

## Geometry Optimization (Positions + Unit Cell)


If you look closely, you will see that the atomic positions updated and the energy went down. The forces went down too.

But note that the unit cell shape and size did not change. In addition to the atomic positions, the unit cell can also be optimized. After all, if you made the unit cell too big, that may not be the lowest energy structure. We can optimize both the positions and unit cell in ASE simultaneously.

In ASE, this is done by wrapping the `Atoms` object in a "Cell Filter". Otherwise, the procedure remains largely the same.


In [20]:
from ase.filters import FrechetCellFilter

# this tells ASE to optimize both atomic positions and cell parameters
fcf = FrechetCellFilter(atoms)


In [21]:
from ase.optimize import BFGS

opt = BFGS(fcf, trajectory="opt2.traj")
opt.run(fmax=0.01)

      Step     Time          Energy          fmax
BFGS:    0 13:53:47     -834.879211        0.099597
BFGS:    1 13:53:47     -834.879639        0.098756
BFGS:    2 13:53:47     -834.890442        0.090780
BFGS:    3 13:53:47     -834.896240        0.094510
BFGS:    4 13:53:47     -834.910645        0.117350
BFGS:    5 13:53:47     -834.924255        0.129470
BFGS:    6 13:53:47     -834.940125        0.116896
BFGS:    7 13:53:47     -834.955383        0.106203
BFGS:    8 13:53:48     -834.971191        0.102707
BFGS:    9 13:53:48     -834.983276        0.095319
BFGS:   10 13:53:48     -834.990906        0.081115
BFGS:   11 13:53:48     -834.996216        0.057151
BFGS:   12 13:53:48     -835.001221        0.066552
BFGS:   13 13:53:48     -835.004822        0.050109
BFGS:   14 13:53:48     -835.006287        0.016701
BFGS:   15 13:53:48     -835.006287        0.005087


np.True_

In [22]:
atoms.get_potential_energy()  # get the optimized energy

-835.0062866210938

In [23]:
view(read("opt2.traj", index=":"))

<Popen: returncode: None args: ['/Users/ct5868/personal/cbe423/.venv/bin/pyt...>

In [24]:
a, b, c = atoms.cell.lengths()
alpha, beta, gamma = atoms.cell.angles()

In [26]:
print(f"a = {a}, b = {b}, c = {c}, alpha = {alpha}, beta = {beta}, gamma = {gamma}")

a = 18.298257510494363, b = 18.298257378145617, c = 18.29825734747982, alpha = 59.99999885454323, beta = 59.99999770837406, gamma = 59.99999805816751


The experimental values for the primitive cell are: a = b = c = 18.303 angstroms and alpha = beta = gamma = 30.0 degrees. Not bad!

For reference, the values for the conventional cell are a = b = c = 25.885 angstroms and alpha = beta = gamma = 90.0 degrees.


## Using Other Calculators


Finally, we will note that there are many ASE calculators, some based on empirical potentials, some based on ML models, some based on quantum chemistry, and so on. An incomplete list can be found at https://ase-lib.org/ase/calculators/calculators.html. It is easy to swap to a different calculator and to do the same procedure as above.

For instance, we could run a calculation with a Lennard-Jones potential in exactly the same way. Note: Of course, the LJ potential we have chosen here is completely unphysical.


In [27]:
from ase.calculators.lj import LennardJones

In [28]:
# We will run a toy single-point calculation with an LJ potential
# This is completely unphysical because we have arbitrarily chosen sigma=1.0 and epsilon=1.0
# But it serves to illustrate how to set up a different calculator and also how
# to specify keyword arguments

atoms.calc = LennardJones(sigma=1.0, epsilon=1.0)
atoms.get_potential_energy()

np.float64(-67.33106460820986)